In [1]:
#Python/Data
import matplotlib.pyplot as plt
from matplotlib import gridspec
import matplotlib.colors as mcolors
import numpy as np
import h5py as h5
import uproot3 as ur
import awkward as ak

from matplotlib import style
#style.use('/global/home/users/ftoralesacosta/dotfiles/scientific.mplstyle')

import os
import shutil

#ML
import energyflow as ef
from energyflow.archs import PFN
from energyflow.utils import data_split
import tensorflow as tf

## File Names and Output Directory

In [2]:
#filename = "2M_uncompressed.hdf5"
#h5_filename = "../generate_data/to_hdf5/Ecut_GeV.hdf5"
#ROOT_file = "../generate_data/to_hdf5/2M_piplus.root"

ROOT_file = "../generate_data/to_hdf5/Uniform_pi+_0-100GeV_standalone.root"
h5_filename = "../generate_data/to_hdf5/Uniform_pi+_0-100GeV_standalone.hdf5"
h5_filename = "../generate_data/to_hdf5/Uniform_pi+_0-100GeV_NO_CELL_CUTS.hdf5"

label = "BugFixed_Data_Insert_NO_MIP_CUT"  #Replace with your own variation!      
path = "./"+label

shutil.rmtree(path, ignore_errors=True)
os.makedirs(path)

## Data Cuts

In [3]:
N_Events = 1_000_000

cut_on_clusters = False
cut_on_genTheta = False
cut_on_genP = False #True for Debug only

min_theta = 5.0 #5 for HCAL
max_theta = 30.0

hit_e_max = 1e10 #Spikes...
hit_e_min = 0.00006 #MIPs
hit_t_max = 200

min_cluster_E = 0.22 #GeV
#min_cluster_E = 0.0 #GeV

#sampling_fraction = 0.02 #HCAL
sampling_fraction = 0.0098 #INSERT

## Model Settings

In [4]:
learning_rate = 1e-5
dropout_rate = 0.05
batch_size = 1000
N_Epochs = 400
patience = 20
N_Latent = 128
shuffle_split = False #Turn FALSE for images!
train_shuffle = False #False for better root hf comparison
Y_scalar = True
loss = 'mae'
#loss = tf.keras.losses.MeanAbsoluteError()

## I/O

In [5]:
h5_file = h5.File(h5_filename,'r')
print(list(h5_file.keys()))

ur_file = ur.open( ROOT_file )
ur_tree = ur_file['events']

['hcal', 'hcali', 'mc']


#### Get HDF5 E hits and Gen P

In [ ]:
h5_detector = 'hcali'
h5_gen_P = h5_file['mc'][:N_Events,8,0]
h5_gen_Theta = h5_file['mc'][:N_Events,9,0]
h5_hits_e = h5_file[h5_detector][:N_Events,0]

#### Get ROOT E hits and Gen P

In [ ]:
#root_detector = "HcalEndcapPHitsReco"
root_detector = "HcalEndcapPInsertHitsReco"

genPx = ur_tree.array('MCParticles.momentum.x',entrystop=N_Events)[:,2]
genPy = ur_tree.array('MCParticles.momentum.y',entrystop=N_Events)[:,2]
genPz = ur_tree.array('MCParticles.momentum.z',entrystop=N_Events)[:,2]
root_gen_P = np.sqrt(genPx*genPx + genPy*genPy + genPz*genPz)
root_gen_Theta = np.arccos(genPz/root_gen_P)*180/np.pi

root_hits_e = ur_tree.array('%s.energy' % root_detector, entrystop=N_Events)
root_hits_t = ur_tree.array('%s.time' % root_detector, entrystop=N_Events)

ROOT needs to have cuts applied here. HDF5 File should already have them applied

In [ ]:
root_cuts = root_hits_e > hit_e_min                                                                   
root_cuts = np.logical_and( root_cuts, root_hits_t <= hit_t_max )  
#root_cuts = np.logical_and( root_cuts, root_hits_e > hit_e_min )                                     



root_hits_e = root_hits_e[root_cuts]

#### Apply HDF5 cuts just in case 

In [ ]:
h5_cuts = h5_hits_e > hit_e_min
#h5_cuts = np.logical_and( h5_cuts, h5_hits_e <= hit_e_max )
h5_cuts = np.logical_and( h5_cuts, h5_hits_e <= hit_t_max )

In [ ]:
from data_functions import make_comparison_plots

In [ ]:
make_comparison_plots( root_hits_e, h5_hits_e, 'Energy' )

In [ ]:
make_comparison_plots( root_gen_P, h5_gen_P, 'GenP' )

In [ ]:
make_comparison_plots( root_gen_Theta, h5_gen_Theta, 'Gen_Theta' )

In [ ]:
print(root_gen_Theta[:6])
print(h5_gen_Theta[:6])

# Cluster Sum

In [ ]:
root_cluster_sum = ak.sum(root_hits_e,axis=-1)
root_cluster_sum = ak.to_numpy(root_cluster_sum)

In [ ]:
hdf5_cluster_sum = np.nansum(h5_hits_e,axis=-1)

## Apply Sampling Fraction (Make sure it's for HCal or Hcal insert!!)

In [ ]:
root_cluster_sum = root_cluster_sum/sampling_fraction
hdf5_cluster_sum = hdf5_cluster_sum/sampling_fraction

# Cut on Clusters

In [ ]:
cluster_mask = root_cluster_sum > min_cluster_E
if (cut_on_genP):
    cluster_mask = np.logical_and(cluster_mask, root_gen_P > 20.)
if (cut_on_clusters):
    root_cluster_sum = root_cluster_sum[cluster_mask]
    hdf5_cluster_sum = hdf5_cluster_sum[cluster_mask]
    root_gen_P = root_gen_P[cluster_mask]
    root_gen_Theta = root_gen_Theta[cluster_mask]
    h5_gen_P = h5_gen_P[cluster_mask]
    h5_gen_Theta = h5_gen_Theta[cluster_mask]

In [ ]:
theta_mask = root_gen_Theta > min_theta
theta_mask = np.logical_and(root_gen_Theta < max_theta,theta_mask)

if (cut_on_genTheta):
    root_cluster_sum = root_cluster_sum[theta_mask]
    hdf5_cluster_sum = hdf5_cluster_sum[theta_mask]
    root_gen_P = root_gen_P[theta_mask]
    root_gen_Theta = root_gen_Theta[theta_mask]
    h5_gen_P = h5_gen_P[theta_mask]
    h5_gen_Theta = h5_gen_Theta[theta_mask]

## Cluster Sum and Truth P Correlation

In [ ]:
from copy import copy
from matplotlib.colors import LogNorm
fig, axes = plt.subplots(nrows=1, ncols=2, figsize=(24, 10), constrained_layout=True)
axes = axes.ravel()
cmap = copy(plt.cm.plasma)
cmap.set_bad(cmap(0))

cluster_edges=np.linspace(0,110,1201)
truth_edges=np.linspace(0,100,1201)


#ROOT
h, xedges, yedges = np.histogram2d(root_gen_P, root_cluster_sum, bins=[truth_edges, cluster_edges])
pcm = axes[0].pcolormesh(xedges, yedges, h.T, cmap=cmap, norm=LogNorm(vmin=1.0e-2,vmax=1.1e1), rasterized=True)
cb = fig.colorbar(pcm, ax=axes[0], pad=0)
cb.ax.tick_params(labelsize=20)
axes[0].set_xlabel("Generated Energy",fontsize=22)
axes[0].set_ylabel("HCal Sum Energy",fontsize=25)
axes[0].set_title("[ROOT] HCal Sum vs. Generated Energy",fontsize=30)
axes[0].plot([0, 100], [0, 100], color="cyan",linewidth=2,alpha=0.5,label="Ideal")
axes[0].legend(loc="upper left")


#HDF5
h, xedges, yedges = np.histogram2d(h5_gen_P, hdf5_cluster_sum, bins=[truth_edges, cluster_edges])
pcm = axes[1].pcolormesh(xedges, yedges, h.T, cmap=cmap, norm=LogNorm(vmin=1.0e-2,vmax=1.1e1), rasterized=True)
cb = fig.colorbar(pcm, ax=axes[1], pad=0)
cb.ax.tick_params(labelsize=20)
axes[1].set_xlabel("Generated Energy",fontsize=22)
axes[1].set_ylabel("HCal Sum Energy",fontsize=25)
axes[1].set_title("[HDF5] HCal Sum vs. Generated Energy",fontsize=30)
axes[1].plot([0, 100], [0, 100], color="cyan",linewidth=2,alpha=0.5,label="Ideal")
axes[1].legend(loc="upper left")

In [ ]:
make_comparison_plots( root_cluster_sum, hdf5_cluster_sum, 'Cluster Sum' )

# At this point, it should be established that ROOT and HDF5 data are IDENTICAL

## Last check: Train on HDF5 and ROOT

In [ ]:
h5_X = hdf5_cluster_sum
h5_Y = h5_gen_P

R_X = root_cluster_sum
R_Y = root_gen_P

In [ ]:
(h5_X_train, h5_X_val, h5_X_test,
h5_Y_train, h5_Y_val, h5_Y_test) = data_split(h5_X, h5_Y, val=0.2, test=0.3,shuffle=shuffle_split)

(R_X_train, R_X_val, R_X_test,
R_Y_train, R_Y_val, R_Y_test) = data_split(R_X, R_Y, val=0.2, test=0.3,shuffle=shuffle_split)

In [ ]:
def step_decay(epoch, lr):
    min_rate = 1.01e-7
    N_start = 10
    N_epochs = 5
    
    return lr
    
    if epoch >= N_start and lr >= min_rate:
        if (epoch%N_epochs==0):
            return lr * 0.1
    return lr

lr_decay = tf.keras.callbacks.LearningRateScheduler(step_decay,verbose=0)

In [ ]:
early_stopping = tf.keras.callbacks.EarlyStopping(patience=patience)

In [ ]:
#from tensorflow.keras.layers import Input
h5_model = tf.keras.models.Sequential([
    tf.keras.layers.Input(shape=[1]),
    tf.keras.layers.Dense(64, activation='relu'), # 1st hiddden layer
    tf.keras.layers.Dense(64, activation='relu'), # 2nd hiddden layer
    tf.keras.layers.Dense(64, activation='relu'), # 3rd hiddden layer
    tf.keras.layers.Dense(64, activation='relu'), # 4th hiddden layer
    tf.keras.layers.Dense(1,activation="linear") # output layer
])

h5_model.compile(loss=loss, optimizer="adam")

h5_fit = h5_model.fit(h5_X_train, h5_Y_train, epochs=100,
                    validation_data=(h5_X_val,h5_Y_val),
                    callbacks=[lr_decay,early_stopping],
                    batch_size=batch_size)


R_model = tf.keras.models.Sequential([
    tf.keras.layers.Input(shape=[1]),
    tf.keras.layers.Dense(64, activation='relu'), # 1st hiddden layer
    tf.keras.layers.Dense(64, activation='relu'), # 2nd hiddden layer
    tf.keras.layers.Dense(64, activation='relu'), # 3rd hiddden layer
    tf.keras.layers.Dense(64, activation='relu'), # 4th hiddden layer
    tf.keras.layers.Dense(1,activation="linear") # output layer
])

R_model.compile(loss=loss, optimizer="adam")

R_fit = R_model.fit(R_X_train, R_Y_train, epochs=100,
                    validation_data=(R_X_val,R_Y_val),
                    callbacks=[lr_decay,early_stopping],
                    batch_size=batch_size)


In [ ]:
h5_model.save("%s/h5_energy_regression.h5"%(path))
h5_preds = h5_model.predict(h5_X_test,batch_size=400)    
np.save("%s/h5_predictions.npy"%(path),h5_preds)
np.save("%s/h5_y_test.npy"%(path),h5_Y_test)
np.save("%s/h5_x_test.npy"%(path),h5_X_test)

R_model.save("%s/R_energy_regression.h5"%(path))
R_preds = R_model.predict(R_X_test,batch_size=400)    
np.save("%s/R_predictions.npy"%(path),R_preds)
np.save("%s/R_y_test.npy"%(path),R_Y_test)
np.save("%s/R_x_test.npy"%(path),R_X_test)

___

In [ ]:
fig, axes = plt.subplots(1, 2,figsize=(28,10),constrained_layout=True)
#fig,axs = plt.subplots(int(N/10),10, figsize=(32, 16),sharex=False,sharey=True,constrained_layout=True)
axes = axes.ravel()
from copy import copy
from matplotlib.colors import LogNorm
#fig, axes = plt.subplots(nrows=1, figsize=(14, 10), constrained_layout=True)
cmap = copy(plt.cm.plasma)
cmap.set_bad(cmap(0))
edges=np.linspace(-10,110,121)
h, xedges, yedges = np.histogram2d(h5_Y_test,h5_preds[:,0], bins=[edges, edges])
#xedges=yedges
pcm = axes[0].pcolormesh(xedges, yedges, h.T, cmap=cmap,
                         norm=LogNorm(vmin=1.0e-2,vmax=2.0e2), rasterized=True)
cb = fig.colorbar(pcm, ax=axes[0], pad=0)
#cb.set_label("Counts",fontsize=22)
cb.ax.tick_params(labelsize=20)
axes[0].set_xlabel("Generated Energy",fontsize=22)
#axes[0].set_yticks(fontsize=20)
#axes[0].set_xticks(fontsize=20)
axes[0].set_ylim(-1,110)
axes[0].set_xlim(-0,101)
axes[0].plot([0, 100], [0, 100], color="cyan",linewidth=2,alpha=0.5,label="Ideal")
axes[0].legend(loc="upper left")
axes[0].set_ylabel("Predicted Energy",fontsize=25)
axes[0].set_title("[HDF5] Predicted vs. Generated Energy",fontsize=30)


axes[1].plot(h5_fit.history['loss'])
axes[1].plot(h5_fit.history['val_loss'])
axes[1].set_title('[HDF5] Model Loss vs. Epoch',fontsize=26)

fig.text(1.05,1.1,label,transform=axes[0].transAxes,fontsize=10)
axes[1].set_ylabel('Mean-Squared Error',fontsize=22)
axes[1].set_yscale('log')
axes[1].set_xlabel('epoch',fontsize=22)
plt.xticks(fontsize=20)
plt.yticks(fontsize=20)
plt.tick_params(direction='in',right=True,top=True,length=10)
plt.tick_params(direction='in',right=True,top=True,which='minor')
axes[1].set_xlim([-1,101])
axes[1].legend(['train', 'validation'], loc='upper right',fontsize=22)
plt.savefig("%s/H5_Correlation.png"%(path))

In [ ]:
fig, axes = plt.subplots(1, 2,figsize=(28,10),constrained_layout=True)
axes = axes.ravel()
from copy import copy
from matplotlib.colors import LogNorm
cmap = copy(plt.cm.plasma)
cmap.set_bad(cmap(0))
edges=np.linspace(-10,110,121)
h, xedges, yedges = np.histogram2d(R_Y_test,R_preds[:,0], bins=[edges, edges])
pcm = axes[0].pcolormesh(xedges, yedges, h.T, cmap=cmap,
                         norm=LogNorm(vmin=1.0e-2,vmax=2.0e2), rasterized=True)
cb = fig.colorbar(pcm, ax=axes[0], pad=0)
cb.ax.tick_params(labelsize=20)
axes[0].set_xlabel("Generated Energy",fontsize=22)
axes[0].set_ylim(-1,110)
axes[0].set_xlim(-0,101)
axes[0].plot([0, 100], [0, 100], color="cyan",linewidth=2,alpha=0.5,label="Ideal")
axes[0].legend(loc="upper left")
axes[0].set_ylabel("Predicted Energy",fontsize=25)
axes[0].set_title("[ROOT] Predicted vs. Generated Energy",fontsize=30)


axes[1].plot(R_fit.history['loss'])
axes[1].plot(R_fit.history['val_loss'])
axes[1].set_title('[ROOT] Model Loss vs. Epoch',fontsize=26)

fig.text(1.05,1.1,label,transform=axes[0].transAxes,fontsize=10)
axes[1].set_ylabel('Mean-Squared Error',fontsize=22)
axes[1].set_yscale('log')
axes[1].set_xlabel('epoch',fontsize=22)
plt.xticks(fontsize=20)
plt.yticks(fontsize=20)
plt.tick_params(direction='in',right=True,top=True,length=10)
plt.tick_params(direction='in',right=True,top=True,which='minor')
axes[1].set_xlim([-1,101])
axes[1].legend(['train', 'validation'], loc='upper right',fontsize=22)
plt.savefig("%s/ROOT_Correlation.png"%(path))

In [ ]:
star_energies = [12,16,20,25,30,50,60,70]
star_res = [0.18, 0.16, 0.15, 0.14, 0.13, 0.098, 0.092, 0.090]

ECCE_res = [0.15,0.127,0.117,0.121,0.106,0.102,0.092,0.098]
ECCE_energies = [10,20,30,40,50,60,80,100]

In [ ]:
from data_functions import get_res_scale

In [ ]:
N_Bins = 20
E_Bins = np.linspace(0,100,N_Bins+1)

h5_resolution, h5_pred_over_truth, h5_avg_truth,h5_slices = get_res_scale(h5_Y_test,h5_preds)
h5_strawman_res, h5_strawman_scale, h5_avg_truth, h5_strawman_slices = get_res_scale(h5_Y_test,h5_X_test)

R_resolution, R_pred_over_truth, R_avg_truth,R_slices = get_res_scale(R_Y_test,R_preds)
R_strawman_res, R_strawman_scale, R_avg_truth,R_strawman_slices = get_res_scale(R_Y_test,R_X_test)

In [ ]:
fig=plt.figure(figsize=(14,10))
plt.title("AI Codesign Resolution",fontsize=25)
plt.ylabel("$(\sigma_{E,\mathrm{Pred}}/E_\mathrm{Truth})$",fontsize=24)
plt.xlabel("$E_\mathrm{Truth}$ [GeV]",fontsize=24)
plt.xticks(fontsize=20)
plt.yticks(fontsize=20)
plt.tick_params(direction='in',right=True,top=True,length=10)
#plt.ylim(-0.02,0.4)
plt.ylim(0,2)
plt.ylim(0,.32)
plt.xlim(-0.01,100.01)
plt.xlim(5.0,100)
#errors = 1.0/(np.sqrt(2*counter-2))*stdev_pred
ax = plt.subplot(1,1,1)
first_bin = 0
last_bin = 20

plt.text(0.8,-0.08,label,transform=ax.transAxes,fontsize=10)
plt.errorbar(h5_avg_truth[first_bin:last_bin],h5_resolution[first_bin:last_bin],#yerr=errors[first_bin:last_bin],
             linestyle="-",linewidth=2.0,capsize=4,capthick=1.2,elinewidth=1.2,ecolor='black',marker="o",color='dodgerblue',alpha=0.7,label="Simple NN [HDF5]")

plt.errorbar(R_avg_truth[first_bin:last_bin],R_resolution[first_bin:last_bin],#yerr=errors[first_bin:last_bin],
             linestyle="-",linewidth=2.0,capsize=4,capthick=1.2,elinewidth=1.2,ecolor='black',marker="o",color='red',alpha=0.7,label="Simple NN [ROOT]")

plt.plot(ECCE_energies,ECCE_res,"-o",label = "EIC Ref",color="limegreen")
plt.plot(star_energies,star_res,"-o",label = "STAR",color="deeppink")
plt.legend(fontsize=15,loc="upper left")
plt.savefig("%s/h5_resolution_plot.pdf"%(path))




In [ ]:
fig=plt.figure(figsize=(14,10))
plt.title("AI Codesign Scale",fontsize=25)
plt.ylabel("$(E_\mathrm{Pred}/E_\mathrm{Truth})$",fontsize=24)
plt.xlabel("$E_\mathrm{Truth}$ [GeV]",fontsize=24)
plt.xticks(fontsize=20)
plt.yticks(fontsize=20)
plt.tick_params(direction='in',right=True,top=True,length=10)
plt.axhline(y=1.0, color='k', linestyle='--',alpha=0.5)#plt.ylim(-0.02,0.4)
plt.ylim(0.9,1.1)
plt.xlim(-1,101)
#errors = 1.0/(np.sqrt(2*counter-2))*stdev_pred
ax = plt.subplot(1,1,1)
first_bin = 0
last_bin = N_Bins

string = "HDF5"
color1 = 'blue'
color2 = 'dodgerblue'
#HDF5 NN   
plt.errorbar(h5_avg_truth[first_bin:last_bin],h5_pred_over_truth[first_bin:last_bin],#yerr=errors[first_bin:last_bin],
             linestyle="--",linewidth=2.0,capsize=4,capthick=1.2,elinewidth=1.2,
             ecolor='black',marker="o",color=color1,alpha=0.7,label="Simple DNN")

#Strawman
plt.errorbar(h5_avg_truth[first_bin:last_bin],h5_strawman_scale[first_bin:last_bin],
             linestyle="-",linewidth=2.0,capsize=4,capthick=1.2,elinewidth=1.2,ecolor='black',
             marker="o",color=color2,alpha=0.7,label="HDF5 Strawman $\sum_\mathrm{Cluster\ E} / %1.2f$"%(sampling_fraction))


#ROOT NN
plt.errorbar(R_avg_truth[first_bin:last_bin],R_pred_over_truth[first_bin:last_bin],#yerr=errors[first_bin:last_bin],
             linestyle="--",linewidth=2.0,capsize=4,capthick=1.2,elinewidth=1.2,
             ecolor='black',marker="o",color='red',alpha=0.7,label="Simple DNN (ROOT)")

#ROOT Strawman

plt.errorbar(R_avg_truth[first_bin:last_bin],R_strawman_scale[first_bin:last_bin],
             linestyle="-",linewidth=2.0,capsize=4,capthick=1.2,elinewidth=1.2,ecolor='black',
             marker="o",color='tomato',alpha=0.7,label="ROOT Strawman $\sum_\mathrm{Cluster\ E} /  $%1.2f"%(sampling_fraction))

#plt.text(0.7,0.7,"ROOT",transform=ax.transAxes,fontsize=25)

plt.legend(fontsize=25)
plt.text(0.8,-0.08,label,transform=ax.transAxes,fontsize=10)


plt.savefig("%s/%s_scale_plot.pdf"%(path,string))

In [ ]:
# 48-52 GeV Slice
#look at the distribution of the PRED in that slice
#
#h5_Y_test,h5_preds

mask = (h5_Y_test < 55.)
mask = np.logical_and(mask,h5_Y_test > 50.)

#h5_X_test = Strawman [HDF5]pre
plt.hist(h5_X_test[mask],bins=50, label="Function Slicing",alpha=0.5)
plt.hist(h5_strawman_slices[10],bins=50,label="Direct Slicing",alpha=0.5)
#plt.hist(h5_strawman_slices[9],bins=50,label="Direct Slicing -1 Bin",alpha=0.5)
#plt.hist(h5_strawman_slices[11],bins=50,label="Direct Slicing +1 Bin",alpha=0.5)
plt.legend()


In [ ]:
fig,axs = plt.subplots(int(N_Bins/10),10, figsize=(32, 10),sharex=False,sharey=True,constrained_layout=True)
for i in range(N_Bins):
    row = int(i/10)
    col = i%10
    if(row==int(N_Bins/10)): continue
    ax = axs[row,col]
    
    if (col==0):
        ax.set_ylabel("Counts",fontsize=15)
    if (np.all(np.isnan(h5_slices[i]))): continue
    temp_bin = np.linspace(h5_avg_truth[i]-2.0,h5_avg_truth[i]+2.0,16)
    ax.set_title("%1.1f $ < E_\mathrm{Truth} < $%1.1f [GeV]"%(E_Bins[i],E_Bins[i+1]),fontsize=10)
    ax.set_xlabel("Predicted Eenergy")
    ax.hist(h5_slices[i],label="Predicted Energies",bins=20)
    ax.axvline(x=h5_avg_truth[i],color='red',alpha=0.3,linestyle="--",label="Avg. $E_\mathrm{Truth} = %1.2f$"%(h5_avg_truth[i]))
    ax.axvline(x=np.nanmean(h5_slices,axis=-1)[i],color='cyan',alpha=0.3,linestyle="--",label="Avg. $E_\mathrm{Pred} = %1.2f$"%(np.nanmean(h5_slices,axis=-1)[i]))

    ax.legend(fontsize=7.5)
    ax.tick_params(direction='in',right=True,top=True,length=5)
plt.savefig("%s/h5_resolutions_slices.pdf"%(path))

In [ ]:
fig,axs = plt.subplots(int(N_Bins/10),10, figsize=(32, 10),sharex=False,sharey=True,constrained_layout=True)
for i in range(N_Bins):
    row = int(i/10)
    col = i%10
    if(row==int(N_Bins/10)): continue
    ax = axs[row,col]
    
    if (col==0):
        ax.set_ylabel("Counts",fontsize=15)

    if (np.all(np.isnan(R_slices[i]))): continue
    temp_bin = np.linspace(R_avg_truth[i]-2.0,R_avg_truth[i]+2.0,16)
    ax.set_title("%1.1f $ < E_\mathrm{Truth} < $%1.1f [GeV]"%(E_Bins[i],E_Bins[i+1]),fontsize=10)
    ax.set_xlabel("Predicted Eenergy")
    ax.hist(R_slices[i],label="Predicted Energies",bins=30)
    ax.axvline(x=R_avg_truth[i],color='red',alpha=0.3,linestyle="--",label="Avg. $E_\mathrm{Truth} = %1.2f$"%(R_avg_truth[i]))
    ax.axvline(x=np.nanmean(R_slices,axis=-1)[i],color='cyan',alpha=0.3,linestyle="--",label="Avg. $E_\mathrm{Pred} = %1.2f$"%(np.nanmean(h5_slices,axis=-1)[i]))

    ax.legend(fontsize=7.5)
    ax.tick_params(direction='in',right=True,top=True,length=5)
plt.savefig("%s/R_resolutions_slices.pdf"%(path))

In [ ]:
fig,axs = plt.subplots(int(N_Bins/10),10, figsize=(32, 10),sharex=False,sharey=True,constrained_layout=True)
for i in range(N_Bins):
    row = int(i/10)
    col = i%10
    if(row==int(N_Bins/10)): continue
    ax = axs[row,col]
    
    if (col==0):
        ax.set_ylabel("Counts",fontsize=15)
    if (np.all(np.isnan(h5_strawman_slices[i]))): continue
    temp_bin = np.linspace(h5_avg_truth[i]-2.0,h5_avg_truth[i]+2.0,16)
    ax.set_title("%1.1f $ < E_\mathrm{Truth} < $%1.1f [GeV]"%(E_Bins[i],E_Bins[i+1]),fontsize=10)
    ax.set_xlabel("Predicted Eenergy")
    ax.hist(h5_slices[i],label="Predicted Energies",bins=30)
    ax.axvline(x=h5_avg_truth[i],color='red',alpha=0.3,linestyle="--",label="Avg. $E_\mathrm{Truth} = %1.2f$"%(h5_avg_truth[i]))
    ax.axvline(x=np.nanmean(h5_strawman_slices,axis=-1)[i],color='cyan',alpha=0.3,linestyle="--",label="Avg. $E_\mathrm{Pred} = %1.2f$"%(np.nanmean(h5_strawman_slices,axis=-1)[i]))

    ax.legend(fontsize=7.5)
    ax.tick_params(direction='in',right=True,top=True,length=5)
plt.savefig("%s/h5_strawman_resolutions_slices.pdf"%(path))

#h5_strawman_res, h5_strawman_scale, h5_avg_truth, h5_strawman_slices

# Let's see what the scale and resolution look like in Bins of Reco (Input) Energy

In [ ]:
reco_h5_resolution, reco_h5_pred_over_truth, reco_h5_avg_reco, reco_h5_avg_truth, reco_h5_slices = get_res_scale_in_reco_bins(h5_Y_test,h5_preds,h5_X_test,22,0,100)
#reco_h5_strawman_res, reco_h5_strawman_scale, reco_h5_avg_truth, reco_h5_strawman_slices = get_res_scale(h5_Y_test,h5_X_test)

reco_R_resolution, reco_R_pred_over_truth, reco_R_avg_reco, reco_R_avg_truth, reco_R_slices = get_res_scale_in_reco_bins(R_Y_test,R_preds,R_X_test,22,0,100)
#R_strawman_res, R_strawman_scale, R_avg_truth,R_strawman_slices = get_res_scale(R_Y_test,R_X_test)

In [ ]:
fig=plt.figure(figsize=(14,10))
plt.title("AI Codesign Scale [Reco Bins]",fontsize=25)
plt.ylabel("$(E_\mathrm{Pred}/E_\mathrm{Truth})$",fontsize=24)
plt.xlabel("$E_\mathrm{RECO}$ [GeV]",fontsize=24)
plt.xticks(fontsize=20)
plt.yticks(fontsize=20)
plt.tick_params(direction='in',right=True,top=True,length=10)
plt.axhline(y=1.0, color='k', linestyle='--',alpha=0.5)#plt.ylim(-0.02,0.4)
plt.ylim(0.8,1.1)
plt.xlim(-1,101)
#errors = 1.0/(np.sqrt(2*counter-2))*stdev_pred
ax = plt.subplot(1,1,1)
first_bin = 0
last_bin = N_Bins

string = "HDF5"
color1 = 'blue'
color2 = 'dodgerblue'
#HDF5 NN   
plt.errorbar(reco_h5_avg_reco[first_bin:last_bin],reco_h5_pred_over_truth[first_bin:last_bin],#yerr=errors[first_bin:last_bin],
             linestyle="--",linewidth=2.0,capsize=4,capthick=1.2,elinewidth=1.2,
             ecolor='black',marker="o",color=color1,alpha=0.7,label="Simple DNN")


#ROOT NN
plt.errorbar(reco_R_avg_reco[first_bin:last_bin],reco_R_pred_over_truth[first_bin:last_bin],#yerr=errors[first_bin:last_bin],
             linestyle="--",linewidth=2.0,capsize=4,capthick=1.2,elinewidth=1.2,
             ecolor='black',marker="o",color='red',alpha=0.7,label="Simple DNN (ROOT)")


#plt.text(0.7,0.7,"ROOT",transform=ax.transAxes,fontsize=25)

plt.legend(fontsize=25)
#plt.text(0.8,-0.08,label,transform=ax.transAxes,fontsize=10)

plt.text(0.65,-0.08,"Reco = $\sum_{E \mathrm{cells}}$",transform=ax.transAxes,fontsize=10)




plt.savefig("%s/Reco_Bins_%s_scale_plot.pdf"%(path,string))

If Normalization is desired